In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [ ]:
# Считываем данные из CSV файла
data = pd.read_csv('data.csv')

# Выбираем признаки и целевую переменную
features = data[['x', 'y', 'z', 'v_x', 'v_y', 'v_z']].values
target = data['pressure'].values

# Разделяем данные на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Определите количество ближайших соседей (в данном случае, 5)
num_neighbors = 5

In [ ]:
# Вычисляем матрицу расстояний между точками обучающей выборки
distances_matrix = kneighbors_graph(features_train, num_neighbors, mode='connectivity').toarray()

# Преобразуем матрицу расстояний в edge_index для обучающей выборки
edge_index_train = torch.tensor(np.vstack(np.where(distances_matrix > 0)), dtype=torch.long)

In [ ]:
features_train_tensor = torch.tensor(features_train, dtype=torch.float)
features_test_tensor = torch.tensor(features_test, dtype=torch.float)
target_train_tensor = torch.tensor(target_train, dtype=torch.float)
target_test_tensor = torch.tensor(target_test, dtype=torch.float)

In [ ]:
data_train = Data(x=features_train_tensor, edge_index=edge_index_train, y=target_train_tensor)
data_test = Data(x=features_test_tensor, edge_index=edge_index_train, y=target_test_tensor)


In [ ]:
lass GNNModel(MessagePassing):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__(aggr='add') # 'add' - агрегационная функция для суммирования сообщений
        
        # Слой для кодирования входных данных
        self.lin_input = nn.Linear(input_dim, hidden_dim)
        
        # GNN слой (передача сообщений)
        self.gnn = nn.GRUCell(hidden_dim, hidden_dim)
        
        # Выходной слой для предсказания pressure
        self.lin_output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # Предобработка данных
        x = self.lin_input(x)
        
        # Добавление петель к рёбрам графа (self-loops)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # Рассылка сообщений и их агрегация
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        return x_j

    def update(self, aggr_out, x):
        # Обновление состояния узлов графа с использованием GNN
        return self.gnn(aggr_out, x)

    def decode(self, x):
        # Предсказание pressure
        x = torch.relu(x)
        x = self.lin_output(x)
        return x

# Определение модели
input_dim = 6  # Размерность входных данных (X, Y, Z, v_x, v_y, v_z)
hidden_dim = 64  # Размерность скрытого слоя
output_dim = 1  # Размерность выхода (pressure)

model = GNNModel(input_dim, hidden_dim, output_dim)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(data_train.x, data_train.edge_index)
    predicted_pressure = model.decode(output)
    loss = criterion(predicted_pressure, data_train.y.view(-1, 1))
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Оценка модели на тестовых данных
model.eval()
with torch.no_grad():
    output_test = model(data_test.x, data_test.edge_index)
    predicted_pressure_test = model.decode(output_test)
    test_loss = criterion(predicted_pressure_test, data_test.y.view(-1, 1))

print(f'Test Loss: {test_loss.item():.4f}')